# RANK WEIGHTED

In [1]:
import pandas as pd
import numpy as np
import pandas_market_calendars as mcal
import warnings
warnings.filterwarnings('ignore')
from empyrical import max_drawdown,cum_returns_final,cum_returns,sortino_ratio,sharpe_ratio,calmar_ratio,annual_return,annual_volatility

import matplotlib.pyplot as plt

import time
import datetime as dt
from datetime import timedelta
from pytz import timezone

import yfinance as yf
from datetime import date
import itertools as it

In [2]:
from platform import python_version

print(python_version())

3.7.4


# LOAD DATA V1

In [3]:
import_path = '/Users/niclaswolnerhanssen/Desktop/LINC/Sentiment LYNX/2_SIGNAL_PROCESSING/'

V1_TWITTER_ZSCORED                      = pd.read_excel(
    import_path+'V1_TWITTER_ZSCORED.xlsx',index_col=0)
V1_REDDIT_ZSCORED                       = pd.read_excel(
    import_path+'V1_REDDIT_ZSCORED.xlsx',index_col=0)
V1_BLOOMBERG_ZSCORED                    = pd.read_excel(
    import_path+'V1_BLOOMBERG_ZSCORED.xlsx',index_col=0)
V1_TWITTER_REDDIT_ZSCORED               = pd.read_excel(
    import_path+'V1_TWITTER_REDDIT_ZSCORED.xlsx',index_col=0)
V1_TWITTER_REDDIT_BLOOMBERG_ZSCORED     = pd.read_excel(
    import_path+'V1_TWITTER_REDDIT_BLOOMBERG_ZSCORED.xlsx',index_col=0)
V1_EMA_ZSCORED                          = pd.read_excel(
    import_path+'V1_EMA_ZSCORED.xlsx',index_col=0)
V1_EMA_TWITTER_ZSCORED                  = pd.read_excel(
    import_path+'V1_EMA_TWITTER_ZSCORED.xlsx',index_col=0)
V1_EMA_REDDIT_ZSCORED                   = pd.read_excel(
    import_path+'V1_EMA_REDDIT_ZSCORED.xlsx',index_col=0)
V1_EMA_BLOOMBERG_ZSCORED                = pd.read_excel(
    import_path+'V1_EMA_BLOOMBERG_ZSCORED.xlsx',index_col=0)
V1_EMA_TWITTER_REDDIT_ZSCORED           = pd.read_excel(
    import_path+'V1_EMA_TWITTER_REDDIT_ZSCORED.xlsx',index_col=0)
V1_EMA_TWITTER_REDDIT_BLOOMBERG_ZSCORED = pd.read_excel(
    import_path+'V1_EMA_TWITTER_REDDIT_BLOOMBERG_ZSCORED.xlsx',index_col=0)
OPEN_TO_CLOSE_STOCK_RETURNS             = pd.read_excel(
    import_path+'V1_OPEN_TO_CLOSE_STOCK_RETURNS.xlsx',index_col=0)
OPEN_TO_CLOSE_EQUALW_RETURNS            = pd.read_excel(
    import_path+'V1_OPEN_TO_CLOSE_EQUALW_RETURNS.xlsx',index_col=0)
OPEN_TO_CLOSE_XLK_RETURNS               = pd.read_excel(
    import_path+'V1_OPEN_TO_CLOSE_XLK_RETURNS.xlsx',index_col=0)
OPEN_TO_OPEN_STOCK_RETURNS              = pd.read_excel(
    import_path+'V1_OPEN_TO_OPEN_STOCK_RETURNS.xlsx',index_col=0)
OPEN_TO_OPEN_EQUALW_RETURNS             = pd.read_excel(
    import_path+'V1_OPEN_TO_OPEN_EQUALW_RETURNS.xlsx',index_col=0)
OPEN_TO_OPEN_XLK_RETURNS                = pd.read_excel(
    import_path+'V1_OPEN_TO_OPEN_XLK_RETURNS.xlsx',index_col=0)

# BACKTEST

$ R_{p} = \sum \limits _{i=1} ^{N} w_{i}r_{i} $

Portfolio return, $R_{p}$, is the sum of the stock-specific return, $r_{i}$, multiplied by the stocks portfolio weight, $w_{i}$. 

### 1. calculate quantile

First we decide where to draw the line for which stocks to long/short

In [4]:
#df = V1_TWITTER_ZSCORED.copy()
signal_df = V1_TWITTER_ZSCORED.copy()
returns_df = OPEN_TO_CLOSE_STOCK_RETURNS.copy()
upper_q = 0.5
lower_q = 0.5
min_stocks_each_basket = 1
extra_weight = 0.2 # in min-max weighting due to min value is removed from the lowest value, 
                   #making the smallest value having zero weight which is a problem
                   # when the number of stocks in each basket is small.

In [5]:
signal_df['RETURNS'] = returns_df['STOCK_RETURNS']

So the idea is to iterate through the dates and calculate the quantile thresholds on each date, calculating the weights then multiply the weights by 0.5 times return for specific stock, then we sum toghether all collected returns to get the porfolio return for that day. 

In [6]:
#create date list
date_list = signal_df['Date'].unique()
feat_name = signal_df.columns[2]

Here is the iteration:

In [7]:
#for i in date_list:
#date_df = df[df['Date']==i]
date_df = signal_df[signal_df['Date']==date_list[-1]]

    

And this is what each days data frame looks like:

In [8]:
date_df

,Date,Stock,TWITTER,RETURNS
16125,2021-11-02,AAPL,-2.653601e-01,0.009148
16126,2021-11-02,ACN,NaN,0.012492
16127,2021-11-02,ADBE,-1.849070e-10,-0.001123
16128,2021-11-02,ADI,6.508390e-01,0.016858
16129,2021-11-02,ADP,-1.711099e-01,0.008645
...,...,...,...,...
16195,2021-11-02,VRSN,NaN,0.008985
16196,2021-11-02,WDC,NaN,0.011767
16197,2021-11-02,WU,NaN,-0.030769
16198,2021-11-02,XLNX,NaN,0.017671


Now we want to divide this date_df by the quantile thresholds of the zscores to a long and short daily df:

In [9]:
#create seperat long/short dfs by quantile
long_date_df  = date_df[date_df[feat_name]>date_df.quantile(q=upper_q)[0]]
short_date_df = date_df[date_df[feat_name]<date_df.quantile(q=lower_q)[0]]

### From here we are making changes from the zscore based allocation

Then some conditions: if there is lower than min_stocks in either basket we do not want to trade, also if there is only 1 stock in either portfolio we want to assign a 0.5 weights to that stock return (50/50 allocation between long/short baskets)

In [10]:
#There must be more stocks than min_stocks in each basket to trade 
if (len(long_date_df)<min_stocks_each_basket)|(len(short_date_df)<min_stocks_each_basket):
    NO_VALUES = True

else:
    NO_VALUES = False
    
if NO_VALUES:
    #append stuff with zeroes:
    #date_list.append(i)
    #total_returns_date_df_list.append(0) 
    print('empty')
    
      

In [11]:
long_date_df

,Date,Stock,TWITTER,RETURNS
16127,2021-11-02,ADBE,-1.849070e-10,-0.001123
16128,2021-11-02,ADI,6.508390e-01,0.016858
16134,2021-11-02,ANET,4.248529e+00,0.004677
16143,2021-11-02,CSCO,5.173483e-16,0.022175
16167,2021-11-02,MA,1.488847e+00,-0.019843
16170,2021-11-02,MSFT,-1.401362e-02,0.008537
16176,2021-11-02,NVDA,9.954677e-01,0.022423
16183,2021-11-02,QCOM,-8.800870e-22,0.005054
16187,2021-11-02,SWKS,0.000000e+00,0.003526


In [12]:
short_date_df

,Date,Stock,TWITTER,RETURNS
16125,2021-11-02,AAPL,-0.265360,0.009148
16133,2021-11-02,AMD,-1.488847,0.021203
16147,2021-11-02,ENPH,-0.974679,-0.001848
16157,2021-11-02,IBM,-0.201113,-0.000950
16158,2021-11-02,INTC,-0.715220,0.005039
16166,2021-11-02,LRCX,-1.077549,-0.000870
16178,2021-11-02,ORCL,-0.409447,0.022832
16182,2021-11-02,PYPL,-0.634620,-0.013245
16190,2021-11-02,TER,-0.537211,0.000000


### weights

Here we do the rank weighting. Notice the -1 multiplication in the short basket, this is to flip the magnitude so that low zscores have the highest short weight and vice versa. 

In [13]:
long_date_df['ZSCORE RANK'] = long_date_df[feat_name].rank()
long_date_df['RANK-WEIGHT'] = long_date_df['ZSCORE RANK']/long_date_df['ZSCORE RANK'].sum()
long_date_df['LONG/SHORT MULTIPLIER'] = np.ones(len(long_date_df)) 

short_date_df['ZSCORE RANK'] = short_date_df[feat_name].mul(-1).rank()
short_date_df['RANK-WEIGHT'] = short_date_df['ZSCORE RANK']/short_date_df['ZSCORE RANK'].sum()
short_date_df['LONG/SHORT MULTIPLIER'] = np.ones(len(short_date_df))*-1

In [14]:
long_date_df.head()

,Date,Stock,TWITTER,RETURNS,ZSCORE RANK,RANK-WEIGHT,LONG/SHORT MULTIPLIER
16127,2021-11-02,ADBE,-1.849070e-10,-0.001123,2.0,0.044444,1.0
16128,2021-11-02,ADI,6.508390e-01,0.016858,6.0,0.133333,1.0
16134,2021-11-02,ANET,4.248529e+00,0.004677,9.0,0.200000,1.0
16143,2021-11-02,CSCO,5.173483e-16,0.022175,5.0,0.111111,1.0
16167,2021-11-02,MA,1.488847e+00,-0.019843,8.0,0.177778,1.0


In [15]:
short_date_df.head()

,Date,Stock,TWITTER,RETURNS,ZSCORE RANK,RANK-WEIGHT,LONG/SHORT MULTIPLIER
16125,2021-11-02,AAPL,-0.265360,0.009148,2.0,0.044444,-1.0
16133,2021-11-02,AMD,-1.488847,0.021203,9.0,0.200000,-1.0
16147,2021-11-02,ENPH,-0.974679,-0.001848,7.0,0.155556,-1.0
16157,2021-11-02,IBM,-0.201113,-0.000950,1.0,0.022222,-1.0
16158,2021-11-02,INTC,-0.715220,0.005039,6.0,0.133333,-1.0


In [16]:
long_date_df['WEIGHTED RETURNS']  = 0.5*long_date_df['RETURNS'] *long_date_df['LONG/SHORT MULTIPLIER'] *long_date_df['RANK-WEIGHT']
short_date_df['WEIGHTED RETURNS'] = 0.5*short_date_df['RETURNS']*short_date_df['LONG/SHORT MULTIPLIER']*short_date_df['RANK-WEIGHT']




In [17]:
long_date_df

,Date,Stock,TWITTER,RETURNS,ZSCORE RANK,RANK-WEIGHT,LONG/SHORT MULTIPLIER,WEIGHTED RETURNS
16127,2021-11-02,ADBE,-1.849070e-10,-0.001123,2.0,0.044444,1.0,-0.000025
16128,2021-11-02,ADI,6.508390e-01,0.016858,6.0,0.133333,1.0,0.001124
16134,2021-11-02,ANET,4.248529e+00,0.004677,9.0,0.200000,1.0,0.000468
16143,2021-11-02,CSCO,5.173483e-16,0.022175,5.0,0.111111,1.0,0.001232
16167,2021-11-02,MA,1.488847e+00,-0.019843,8.0,0.177778,1.0,-0.001764
16170,2021-11-02,MSFT,-1.401362e-02,0.008537,1.0,0.022222,1.0,0.000095
16176,2021-11-02,NVDA,9.954677e-01,0.022423,7.0,0.155556,1.0,0.001744
16183,2021-11-02,QCOM,-8.800870e-22,0.005054,3.0,0.066667,1.0,0.000168
16187,2021-11-02,SWKS,0.000000e+00,0.003526,4.0,0.088889,1.0,0.000157


In [18]:
short_date_df

,Date,Stock,TWITTER,RETURNS,ZSCORE RANK,RANK-WEIGHT,LONG/SHORT MULTIPLIER,WEIGHTED RETURNS
16125,2021-11-02,AAPL,-0.265360,0.009148,2.0,0.044444,-1.0,-0.000203
16133,2021-11-02,AMD,-1.488847,0.021203,9.0,0.200000,-1.0,-0.002120
16147,2021-11-02,ENPH,-0.974679,-0.001848,7.0,0.155556,-1.0,0.000144
16157,2021-11-02,IBM,-0.201113,-0.000950,1.0,0.022222,-1.0,0.000011
16158,2021-11-02,INTC,-0.715220,0.005039,6.0,0.133333,-1.0,-0.000336
16166,2021-11-02,LRCX,-1.077549,-0.000870,8.0,0.177778,-1.0,0.000077
16178,2021-11-02,ORCL,-0.409447,0.022832,3.0,0.066667,-1.0,-0.000761
16182,2021-11-02,PYPL,-0.634620,-0.013245,5.0,0.111111,-1.0,0.000736
16190,2021-11-02,TER,-0.537211,0.000000,4.0,0.088889,-1.0,-0.000000


Time for wrapping things up!

In [19]:
#only on trading days (if min_stock is fulfilled)
long_returns_date_sum = long_date_df['WEIGHTED RETURNS'].sum()#append
short_returns_date_sum = short_date_df['WEIGHTED RETURNS'].sum()#append
total_returns_date_sum = long_returns_date_sum+short_returns_date_sum#append
diagnostics_df = pd.concat([long_date_df,short_date_df]).sort_index()#append

In [20]:
diagnostics_df

,Date,Stock,TWITTER,RETURNS,ZSCORE RANK,RANK-WEIGHT,LONG/SHORT MULTIPLIER,WEIGHTED RETURNS
16125,2021-11-02,AAPL,-2.653601e-01,0.009148,2.0,0.044444,-1.0,-0.000203
16127,2021-11-02,ADBE,-1.849070e-10,-0.001123,2.0,0.044444,1.0,-0.000025
16128,2021-11-02,ADI,6.508390e-01,0.016858,6.0,0.133333,1.0,0.001124
16133,2021-11-02,AMD,-1.488847e+00,0.021203,9.0,0.200000,-1.0,-0.002120
16134,2021-11-02,ANET,4.248529e+00,0.004677,9.0,0.200000,1.0,0.000468
16143,2021-11-02,CSCO,5.173483e-16,0.022175,5.0,0.111111,1.0,0.001232
16147,2021-11-02,ENPH,-9.746794e-01,-0.001848,7.0,0.155556,-1.0,0.000144
16157,2021-11-02,IBM,-2.011126e-01,-0.000950,1.0,0.022222,-1.0,0.000011
16158,2021-11-02,INTC,-7.152199e-01,0.005039,6.0,0.133333,-1.0,-0.000336
16166,2021-11-02,LRCX,-1.077549e+00,-0.000870,8.0,0.177778,-1.0,0.000077


# Throwing all into a function

In [21]:
def backtest_rank(signal_df,returns_df,upper_q=0.5,lower_q = 0.5,min_stocks_each_basket = 1):
    signal_df['RETURNS'] = returns_df['STOCK_RETURNS']
    #create date list
    date_list = signal_df['Date'].unique()
    feat_name = signal_df.columns[2]
    
    DATES = []
    TRADE_DATES = []
    MKT_NEUTRAL_RETURNS = []
    LONG_RETURNS = []
    SHORT_RETURNS = []
    DIAGNOSTICS = []
    
    
    for i in date_list:
        date_df = signal_df[signal_df['Date']==i]

        #create seperat long/short dfs by quantile
        long_date_df  = date_df[date_df[feat_name]>date_df.quantile(q=upper_q)[0]]
        short_date_df = date_df[date_df[feat_name]<date_df.quantile(q=lower_q)[0]]
        
        #There must be more stocks than min_stocks in each basket to trade 
        if (len(long_date_df)<min_stocks_each_basket)|(len(short_date_df)<min_stocks_each_basket):
            NO_VALUES = True

        else:
            NO_VALUES = False

        if NO_VALUES==True:
            DATES.append(i)
            MKT_NEUTRAL_RETURNS.append(0) 
            continue
            
        ######## RANK WEIGHTS
        
        #SHORT
        short_date_df['ZSCORE RANK'] = short_date_df[feat_name].mul(-1).rank()
        short_date_df['RANK-WEIGHT'] = short_date_df['ZSCORE RANK']/short_date_df['ZSCORE RANK'].sum()
        short_date_df['LONG/SHORT MULTIPLIER'] = np.ones(len(short_date_df))*-1

        #LONG
        long_date_df['ZSCORE RANK'] = long_date_df[feat_name].rank()
        long_date_df['RANK-WEIGHT'] = long_date_df['ZSCORE RANK']/long_date_df['ZSCORE RANK'].sum()
        long_date_df['LONG/SHORT MULTIPLIER'] = np.ones(len(long_date_df))  
        
        ########RETURNS
        
        long_date_df['WEIGHTED RETURNS']  = 0.5*long_date_df['RETURNS'] *long_date_df['LONG/SHORT MULTIPLIER'] *long_date_df['RANK-WEIGHT']
        short_date_df['WEIGHTED RETURNS'] = 0.5*short_date_df['RETURNS']*short_date_df['LONG/SHORT MULTIPLIER']*short_date_df['RANK-WEIGHT']
    
        ######APPEND
        
        #only on trading days (if min_stock is fulfilled)
        long_returns_date_sum = long_date_df['WEIGHTED RETURNS'].sum()#append
        LONG_RETURNS.append(long_returns_date_sum)
        
        short_returns_date_sum = short_date_df['WEIGHTED RETURNS'].sum()#append
        SHORT_RETURNS.append(short_returns_date_sum)
        
        total_returns_date_sum = long_returns_date_sum+short_returns_date_sum#append
        MKT_NEUTRAL_RETURNS.append(total_returns_date_sum)
        
        diagnostics_df = pd.concat([long_date_df,short_date_df]).sort_index()#append
        DIAGNOSTICS.append(diagnostics_df)
        
        TRADE_DATES.append(i)
        DATES.append(i)
        
    ##########pile to dfs
    
    STRATEGY_RETURNS            = pd.DataFrame(index=DATES)
    STRATEGY_RETURNS[feat_name] = MKT_NEUTRAL_RETURNS
    
    LS_RETURNS = pd.DataFrame(index=TRADE_DATES)
    LS_RETURNS[feat_name+'_LONG_RETURNS']  = LONG_RETURNS
    LS_RETURNS[feat_name+'_SHORT_RETURNS'] = SHORT_RETURNS
    
    #LS_returns only includes when returns when trading, so cumulative rets are going to be wrong, lets fix:
    LS_RETURNS = pd.concat([STRATEGY_RETURNS,LS_RETURNS],axis=1)
    LS_RETURNS = LS_RETURNS.drop(feat_name,axis=1)
    LS_RETURNS = LS_RETURNS.replace(np.nan,0)
    
    DIAGNOSTICS_DF = pd.concat(DIAGNOSTICS)
    
    return STRATEGY_RETURNS, LS_RETURNS, DIAGNOSTICS_DF
    
    

In [22]:
STRATEGY_RETURNS, LS_RETURNS, DIAGNOSTICS_DF = backtest_rank(signal_df  = signal_df
              ,returns_df= returns_df
              ,upper_q   = upper_q
              ,lower_q   = lower_q
              ,min_stocks_each_basket = min_stocks_each_basket)

In [23]:
DIAGNOSTICS_DF[DIAGNOSTICS_DF['Date']=='2021-11-02']

,Date,Stock,TWITTER,RETURNS,ZSCORE RANK,RANK-WEIGHT,LONG/SHORT MULTIPLIER,WEIGHTED RETURNS
16125,2021-11-02,AAPL,-2.653601e-01,0.009148,2.0,0.044444,-1.0,-0.000203
16127,2021-11-02,ADBE,-1.849070e-10,-0.001123,2.0,0.044444,1.0,-0.000025
16128,2021-11-02,ADI,6.508390e-01,0.016858,6.0,0.133333,1.0,0.001124
16133,2021-11-02,AMD,-1.488847e+00,0.021203,9.0,0.200000,-1.0,-0.002120
16134,2021-11-02,ANET,4.248529e+00,0.004677,9.0,0.200000,1.0,0.000468
16143,2021-11-02,CSCO,5.173483e-16,0.022175,5.0,0.111111,1.0,0.001232
16147,2021-11-02,ENPH,-9.746794e-01,-0.001848,7.0,0.155556,-1.0,0.000144
16157,2021-11-02,IBM,-2.011126e-01,-0.000950,1.0,0.022222,-1.0,0.000011
16158,2021-11-02,INTC,-7.152199e-01,0.005039,6.0,0.133333,-1.0,-0.000336
16166,2021-11-02,LRCX,-1.077549e+00,-0.000870,8.0,0.177778,-1.0,0.000077


In [24]:
diagnostics_df

,Date,Stock,TWITTER,RETURNS,ZSCORE RANK,RANK-WEIGHT,LONG/SHORT MULTIPLIER,WEIGHTED RETURNS
16125,2021-11-02,AAPL,-2.653601e-01,0.009148,2.0,0.044444,-1.0,-0.000203
16127,2021-11-02,ADBE,-1.849070e-10,-0.001123,2.0,0.044444,1.0,-0.000025
16128,2021-11-02,ADI,6.508390e-01,0.016858,6.0,0.133333,1.0,0.001124
16133,2021-11-02,AMD,-1.488847e+00,0.021203,9.0,0.200000,-1.0,-0.002120
16134,2021-11-02,ANET,4.248529e+00,0.004677,9.0,0.200000,1.0,0.000468
16143,2021-11-02,CSCO,5.173483e-16,0.022175,5.0,0.111111,1.0,0.001232
16147,2021-11-02,ENPH,-9.746794e-01,-0.001848,7.0,0.155556,-1.0,0.000144
16157,2021-11-02,IBM,-2.011126e-01,-0.000950,1.0,0.022222,-1.0,0.000011
16158,2021-11-02,INTC,-7.152199e-01,0.005039,6.0,0.133333,-1.0,-0.000336
16166,2021-11-02,LRCX,-1.077549e+00,-0.000870,8.0,0.177778,-1.0,0.000077


Sems right!